In [56]:
import cv2,os
import numpy as np

In [57]:
# distance
def distance(x1,x2):
    return np.sqrt(np.sum((x1-x2)**2))


def knn(X,Y,query_point,k=6):
    m=X.shape[0]
    l=[]
    for i in range(m):
        dist=distance(query_point,X[i])
        l.append((dist,Y[i]))
    val=sorted(l)
    val=val[:k]
    val=np.array(val)
    r=np.unique(val[:,1],return_counts=True)
    idx=np.argmax(r[1])
    return r[0][idx]
    

In [58]:
data_path='DATA/'
label_id=0
labels=[]
data=[]
names={}

In [59]:
for fx in os.listdir(data_path):
    if fx.endswith('.npy'):
        d=np.load(data_path+fx)
        names[label_id]=fx[:-4]
        data.append(d)
        label=label_id*np.ones((d.shape[0],))
        labels.append(label)
        label_id+=1
data=np.concatenate(data,axis=0)        
labels=np.concatenate(labels,axis=0).reshape((-1,1))

In [60]:
cap=cv2.VideoCapture(0)
face_finder=cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [61]:
while True:
    ret,frame=cap.read()
    if ret==False:
        continue
    faces=face_finder.detectMultiScale(frame,1.3,5)    
    for (x,y,w,h) in faces:
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100))
        out=knn(data,labels,face_section)
        pred_name=names[int(out)]
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2) 
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_COMPLEX, 1,(255,0,0),2,cv2.LINE_AA)
    cv2.imshow("image",frame)    
    key=cv2.waitKey(1) & 0xFF
    if key==ord('q') :
        break
cap.release()
cv2.destroyAllWindows()        
        